# Proceso de extraccion , transformacion y carga (ETL) CSV a BD SQL Server

## Preparativos

Importacion de librerias necesarias


In [1]:
import pyodbc
from dotenv import load_dotenv
import os
import zipfile
import pandas as pd

Cargar las variables de entorno desde el .env

In [2]:
load_dotenv()

True

## Extracción

### Conexiona a SQL Server

Obtener variables del entorno

In [3]:
server = os.getenv('DB_SERVER')
database = os.getenv('DB_DATABASE')
username = os.getenv('DB_USERNAME')
password = os.getenv('DB_PASSWORD')

Verificar que las variables se cargaron

In [4]:
print(f"Conectando a {database} en {server}")

Conectando a ETL1 en ALEJANDRO


In [8]:
#cadena de conexion
conn_str  = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

Prueba de conexion

In [9]:

try:
    conn = pyodbc.connect(conn_str)
    print("Conexión exitosa")
    conn.close()
except Exception as e:
    print(f"Error: {e}")

Conexión exitosa


Generamos la variables cursor

In [10]:
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

### Carga de Archivo CSV 

El csv se encuentra en un archivo Zip por lo cual se debe descomprimir

In [27]:
# Obtén la ruta del archivo ZIP desde una variable de entorno
zip_file_path = os.getenv('ZIP_FILE_PATH')

# Abre el archivo ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Lista de archivos en el ZIP
    file_list = zip_ref.namelist()
    
    # Asegúrate de que el archivo deseado esté en el ZIP
    if len(file_list) > 0:
        # Nombre del primer archivo en el ZIP
        csv_file_name = file_list[0]
        
        # Lee el archivo CSV directamente desde el ZIP
        with zip_ref.open(csv_file_name) as csv_file:
            df = pd.read_csv(csv_file)
    else:
        print("El archivo ZIP no contiene archivos.")

## Transformacion de datos

### Informacion de la base de datos


La base de datos esta en modelo estrella como eje central se encuentra la tabla la tabla Shows

In [17]:
cursor.execute("SELECT * FROM Shows")
columnas = [descripcion[0] for descripcion in cursor.description]
print(columnas)

['Show_id', 'Title', 'Country', 'Rating', 'Director', 'Cast_id', 'Type_show_id', 'Caracteristic_id', 'Date_show_added_id', 'Date_show_release_id']


y las tablas que la rodean y conformar la forma de estra son Casts, date_shows_added, Caracteristics, Date_show_release,Type_Show. 
*La duracion se descarto de la tabla que en este caso se estan evaluando peliculas y series, donde las series se miden en temporadas y las peliculas en minutos y al no conocer el tiempo por capitulos o capitulos por temporadas no se puede controlar la columna y generar data de valor para este modelo en especifico, en otro modelo de BD es una columna con mucho valor.

In [20]:
cursor.execute("SELECT * FROM Date_show_added")
columnas = [descripcion[0] for descripcion in cursor.description]
print(columnas)

['Date_show_added_id', 'Fecha', 'Year_added', 'Month_added', 'Day_added']


In [21]:
cursor.execute("SELECT * FROM Caracteristics")
columnas = [descripcion[0] for descripcion in cursor.description]
print(columnas)

['Caracteristic_id', 'Listen_id', 'Description_caract']


In [22]:
cursor.execute("SELECT * FROM Date_show_release")
columnas = [descripcion[0] for descripcion in cursor.description]
print(columnas)

['Date_show_release_id', 'Year_release']


In [24]:
cursor.execute("SELECT * FROM casts")
columnas = [descripcion[0] for descripcion in cursor.description]
print(columnas)

['Cast_id', 'Name_cast']


In [25]:
cursor.execute("SELECT * FROM type_show")
columnas = [descripcion[0] for descripcion in cursor.description]
print(columnas)

['Type_show_id', 'Type_show']


### Transformacion Datos CSV 

Generamos una vista previa de los datos del CSV

In [ ]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


Identificamos las columasn 

In [ ]:
df.columns

Index(['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description'],
      dtype='object')

La cantidad de filas y columnas

In [ ]:
df.shape

(8807, 12)

Y los tipos de dato de cada columna

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


Realizaremos control de datos nulos

In [28]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

Como se indico no se contara con la columna Duration.
Primero se generara una variable de trabajo

In [32]:
df1 = df

In [33]:
df1 = df1.drop('duration', axis=1)

En el caso de los directores, se asignaran los valores nullos con 'desconocido'

In [34]:
df1['director'].fillna('Desconocido', inplace=True)